### Imports

In [ ]:
import os, json
import pandas as pd
import numpy as np
from geopy import distance
from datetime import datetime, timedelta

import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Functions

In [ ]:
base_path = '/content/drive/MyDrive/complexsystems/'

In [ ]:
def read_files(path):
  dfs = []

  for year in ['2018','2019']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
      
      try:
        tmp = pd.read_csv(path + 'data/raw/' + year + month + '-fordgobike-tripdata.csv', low_memory=False)
      except:
        tmp = pd.read_csv(path + 'data/raw/' + year + month + '-baywheels-tripdata.csv', low_memory=False)

      dfs.append(tmp)

  concat_df = pd.concat(dfs).reset_index(drop=True)
  return concat_df

In [ ]:
##### PER DATI POST COVID, METTE NOME CITTA'
# # tiene solo stazioni valide (toglie stazioni temporanee e robe strane)
# codes = {'SF': 'san francisco', 'BK': 'oakland', 'OK': 'oakland', 'EM': 'oakland', 'SJ': 'san jose'}
# ids = set(list(df['start_station_id']) + list(df['end_station_id']))
# valid_ids = [_id for _id in ids if _id[:2] in codes]
# df = df[df['start_station_id'].isin(valid_ids)]
# df = df[df['end_station_id'].isin(valid_ids)]
# 
# # aggiunta colonna città
# df['city'] = df.apply(lambda x: codes[x['start_station_id'][:2]] if x['start_station_id'][:2] == x['end_station_id'][:2] else None, axis=1)
# df = df.copy().dropna()

In [ ]:
def preprocess(df):

  # rename columns
  df.rename(columns={'start_time': 'time1', 
                     'end_time': 'time2', 
                     'start_station_id': 'id1', 
                     'end_station_id': 'id2', 
                     'start_station_name': 'name1', 
                     'end_station_name': 'name2', 
                     }, inplace=True)

  # NaN values
  for c in ['time1', 'time2', 'id1', 'id2']: 
    df = df[df[c].notna()]

  # ids as integer
  for c in ['id1', 'id2']:
    df[c] = df[c].astype(int)

  # lowercase user type
  df['user_type'] = df['user_type'].str.lower()

  # trip duration
  df['time1'] = pd.to_datetime(df['time1'])
  df['time2'] = pd.to_datetime(df['time2'])
  df['duration'] = (df['time2'] - df['time1']).astype('timedelta64[s]') / 60
  
  # trips must last between 1 minute and 12 hours, otherwise it's an error / test [ASSUMPTION]
  df = df[(df['duration'] > 1) & (df['duration'] < 720)]

  # merge coordinates
  df['coords1'] = list(zip(df['start_station_latitude'], df['start_station_longitude']))
  df['coords2'] = list(zip(df['end_station_latitude'], df['end_station_longitude']))

  # delete coordinates errors
  df = df[df['coords1']!=(0,0)]
  df = df[df['coords2']!=(0,0)]

  # define cities
  coords_set = set(df['coords1']).union(set(df['coords2']))
  d = {c: find_city(c) for c in coords_set}
  df['city'] = df['coords1'].apply(lambda coords: d[coords])
  df['city2'] = df['coords2'].apply(lambda coords: d[coords])
  df = df[df['city']==df['city2']] # ELIMINO CIRCA 200 RIGHE, DI VIAGGI TRA CITTA' DIVERSE

  # for each station, avg of coordinates if not accurate
  tmp1 = df[['id1', 'coords1']]
  tmp2 = df[['id2', 'coords2']]
  tmp2.columns = ['id1', 'coords1']
  tmp = pd.concat([tmp1, tmp2])
  d = {}
  for i in set(tmp['id1']):
    tmp2 = tmp[tmp['id1']==i].copy()
    s = set(tmp2['coords1'])
    if len(s) != 1:
      d[i] = tuple(np.mean(list(s), axis=0))
    else:
      d[i] = list(s)[0]
  df['coords1'] = df['id1'].apply(lambda coords: d[coords])
  df['coords2'] = df['id2'].apply(lambda coords: d[coords])
  
  # reformat 
  df = df.reset_index(drop=True)
  df = df[['time1', 'time2', 'id1', 'id2', 'name1', 'name2', 'coords1', 'coords2', 'duration', 'city', 'bike_id', 'user_type']]

  # exlude fake stations
  df = df[(df['id1']!=408) & (df['id1']!=484) & (df['id2']!=408) & (df['id2']!=484)]

  return df

In [ ]:
def find_city(coords):
  city_centers = {(37.770436, -122.431766): 'san francisco', (37.355716, -121.898813): 'san jose', (37.805503, -122.272598): 'oakland'}
  dist = [distance.distance(c, coords).kilometers for c in city_centers]
  closest = np.argmin(dist)
  return city_centers[list(city_centers.keys())[closest]]

### Read & preprocess files

In [ ]:
concat_df = read_files(base_path)

In [ ]:
bike_df = preprocess(concat_df)
bike_df

,time1,time2,id1,id2,name1,name2,coords1,coords2,duration,city,bike_id,user_type
0,2018-01-31 19:52:24.667,2018-02-01 06:58:31.053,74,19,Laguna St at Hayes St,Post St at Kearny St,"(37.7764348192, -122.4262440205)","(37.788975, -122.403452)",666.100000,san francisco,617,subscriber
1,2018-01-31 22:58:44.867,2018-02-01 00:46:41.908,236,160,Market St at 8th St,West Oakland BART Station,"(37.8036865, -122.282497)","(37.8053183, -122.2948365)",107.950000,oakland,1306,customer
2,2018-01-31 23:53:53.632,2018-02-01 00:01:26.805,110,134,17th & Folsom Street Park (17th St at Folsom St),Valencia St at 24th St,"(37.7637085, -122.4152042)","(37.7524278, -122.4206278)",7.550000,san francisco,3571,subscriber
3,2018-01-31 23:52:09.903,2018-01-31 23:55:10.807,81,93,Berry St at 4th St,4th St at Mission Bay Blvd S,"(37.77588, -122.39317)","(37.7704074, -122.3911984)",3.000000,san francisco,1403,subscriber
4,2018-01-31 23:34:56.004,2018-01-31 23:51:32.674,134,4,Valencia St at 24th St,Cyril Magnin St at Ellis St,"(37.7524278, -122.4206278)","(37.7858806269, -122.4089150084)",16.600000,san francisco,3675,subscriber
...,...,...,...,...,...,...,...,...,...,...,...,...
4255805,2019-12-30 10:10:18.000,2019-12-30 10:24:31.000,6,5,The Embarcadero at Sansome St,Powell St BART Station (Market St at 5th St),"(37.80477, -122.403234)","(37.7838993571, -122.4084448814)",14.216667,san francisco,104326,subscriber
4255806,2019-12-19 16:33:10.000,2019-12-19 16:39:35.000,6,20,The Embarcadero at Sansome St,Mechanics Monument Plaza (Market St at Bush St),"(37.80477, -122.403234)","(37.7913, -122.399051)",6.416667,san francisco,829584,subscriber
4255807,2019-12-31 20:16:15.000,2019-12-31 20:42:37.000,6,50,The Embarcadero at Sansome St,2nd St at Townsend St,"(37.80477, -122.403234)","(37.780526, -122.390288)",26.366667,san francisco,882336,subscriber
4255808,2019-12-31 10:58:30.000,2019-12-31 11:27:10.000,6,371,The Embarcadero at Sansome St,Lombard St at Columbus Ave,"(37.80477, -122.403234)","(37.8027461525, -122.4135786295)",28.666667,san francisco,655789,subscriber


### Save

In [ ]:
for city in set(bike_df['city']): 
  tmp = bike_df[bike_df['city']==city].copy()
  del tmp['city']
  name = city.replace(' ', '_')
  tmp.to_csv(base_path+'data/preprocessed/preprocessed_data_'+name+'.csv', index=False)